In [1]:
# Import required libraries
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

# Define the pre-trained model to be used
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

# Initialize the tokenizer using the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Initialize a causal language model using the specified pre-trained model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Set up configuration for text generation
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512  # Set the maximum number of tokens to generate
generation_config.temperature = 0.0001  # Set the randomness in sampling
generation_config.top_p = 0.95  # Control diversity in generated text
generation_config.do_sample = True  # Use sampling during text generation
generation_config.repetition_penalty = 1.15  # Penalize repeated tokens

# Create a text generation pipeline with specified model, tokenizer, and configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

# Initialize a custom pipeline possibly with additional model arguments
# In this case, setting temperature to 0 for the text generation pipeline
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Check the prompting through sample
%%time
prompt = "[INST] What is the capital of India? [/INST]"
sample = llm(prompt)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 3.1 s, sys: 176 ms, total: 3.27 s
Wall time: 3.28 s


In [3]:
sample

' The capital city of India is New Delhi.'

In [4]:
def extract_data(data):
    # Initialize empty lists to store extracted data
    contexts = []     # Store contexts from paragraphs
    questions = []    # Store questions from Q&A pairs
    answers = []  # Store lists of answers corresponding to questions
    ids = []        # Store IDs of questions
    
    # Loop through the provided data
    for row in data:
        # Extract paragraphs within the data
        for paragraph in row['paragraphs']:
            # Extract Q&A pairs within each paragraph
            for qas in paragraph['qas']:
                # Store context for each Q&A pair
                contexts.append(paragraph['context'])
                
                # Store the question itself
                questions.append(qas['question'])
                
                # Store all answers related to the question
                answer_texts = []
                for answer in qas['answers']:
                    answer_texts.append(answer['text'])
                answers.append(answer_texts)
                
                # Store the ID of the question
                ids.append(qas['id'])
    
    # Return the extracted data
    return contexts, questions, answers, ids


In [5]:
path = "data/train_webmd_squad_v2_full.json"
def load_data(path):
    # Load JSON data from the given path
    obj = json.load(open(path))
    
    # Extract the 'data' field from the loaded JSON object
    obj = obj['data']
    
    # Process the extracted data to get context, question, answers, and IDs
    context, question, answers, ids = extract_data(obj)
    
    # Create a Dataset object using the processed data
    data = Dataset.from_dict({
        'context': context,
        'question': question,
        'answer': answers,
        'id': ids
    })
    
    # Return the constructed Dataset object
    return data


In [6]:
from datasets import Dataset
import json

# Load the data
health_data = load_data(path)


In [7]:
health_data['context'][18]

"Migraine with brainstem aura or MBA (formerly known as basilar migraines) are headaches that start in the lower part of the brain, called the brainstem. They cause symptoms such as dizziness, double vision, and lack of coordination. These changes, called an aura, can happen about 10 minutes to 45 minutes before your head hurts. The headache pain of a basilar migraine often starts on one side of the head and then gradually spreads and gets stronger. This type of migraine can last anywhere from 4 to 72 hours. And it takes time to recover from one. You may feel drained for up to 24 hours after it's over. Migraines with brainstem aura are known by several different names: Basilar migraine Basilar artery migraine Basilar-type migraine Bickerstaff's syndrome Brainstem migraine Vertebrobasilar migraine MBA can affect people of all ages. Generally, though, they start in childhood or the teen years. Women are slightly more likely to have them than men. Triggers may include: Alcohol Stress Lack

In [8]:
health_data['question'][18]

'Can migraine with brainstem aura be prevented?'

In [9]:
health_data['answer'][18]

['To keep from getting a MBA, it helps to avoid the things that usually cause one. Keep a journal of your attacks so you can figure out the things that trigger them. It also helps to live a healthy lifestyle. That means you need to: Get enough sleep.']

## A. ZERO SHOT

In [10]:
from langchain import PromptTemplate

template = """
[INST] <>
Act as a helpful assistant and answer the user's questions. Do not give any false information. In case you don't have answer, specify why the question can't be answered.
<>

{question} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)
     

In [11]:
%%time

question = "What types of exercise are best for people with asthma?"

result = llm(prompt.format(question=question))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


There are several types of exercises that can be beneficial for people with asthma. Low-impact activities such as swimming, cycling, or walking can help improve cardiovascular fitness without triggering an asthma attack. Yoga and tai chi may also be helpful in improving flexibility and reducing stress levels. It is important to consult with a healthcare provider before starting any new exercise routine, especially if you have asthma, to ensure that it is safe and appropriate for your individual needs.
CPU times: user 6.67 s, sys: 2.56 ms, total: 6.67 s
Wall time: 6.67 s


In [12]:
question = " How is obsessive-compulsive disorder diagnosed? "

result = llm(prompt.format(question=question))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Obsessive-compulsive disorder (OCD) is typically diagnosed through a combination of self-reporting symptoms, clinical interview, and sometimes psychological testing. During the diagnostic process, healthcare providers will assess the severity and duration of symptoms, as well as their impact on daily life. They may also rule out other conditions that might present with similar symptoms. While there isn't a specific test to diagnose OCD, it's important to note that a proper diagnosis can help individuals receive appropriate treatment and support.


In [13]:
%%time
question = " When are you more likely to get a blood clot? "

result = llm(prompt.format(question=question))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A blood clot is most likely to occur when there is a blockage in one of your arteries or veins. This can happen due to various reasons such as injury, infection, inflammation, or certain medical conditions like heart disease, diabetes, or cancer. It can also occur during surgery or after immobility for long periods of time. However, it's important to note that anyone can get a blood clot at any time, regardless of their age or health status. If you have concerns about your risk of getting a blood clot, it's best to consult with a healthcare professional.
CPU times: user 8.22 s, sys: 6.97 ms, total: 8.23 s
Wall time: 8.22 s


In [14]:
%%time
question = " How should you lift objects to prevent back pain? "

result = llm(prompt.format(question=question))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Lifting objects correctly is important for preventing back pain. Here are some tips on how to lift objects safely:

1. Warm up before lifting: Before attempting to lift an object, do some light stretching or cardio exercise to warm up your muscles. This will help prevent injury and make it easier to lift the object.
2. Bend at the knees: When lifting an object, bend at the knees rather than at the waist. This will help distribute the weight of the object evenly across your body and reduce strain on your back.
3. Keep your back straight: As you lift the object, keep your back straight. Avoid arching your back or slouching.
4. Use proper lifting techniques: Depending on the size and weight of the object, use the appropriate lifting technique. For example, if you need to lift something heavy, consider using a hand truck or dolly to move it.
5. Take breaks: If you need to lift multiple objects, take breaks frequently to rest your back and avoid overexertion.
6. Seek assistance: If you find

In [15]:
%%time
question = " How can you be smart with antibiotics? "

result = llm(prompt.format(question=question))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Antibiotics are powerful medications that can help fight bacterial infections. To use them safely and effectively, it is important to follow your doctor's instructions carefully. Here are some tips for being smart with antibiotics:

1. Take the full course of treatment: Antibiotics work by killing bacteria over time. It is important to take the full course of treatment prescribed by your doctor to ensure that all the bacteria are killed and the infection is fully cleared.
2. Don't skip doses: Skipping doses can allow the remaining bacteria to multiply and become resistant to the antibiotic, making it less effective.
3. Take the medication at the right times: Antibiotics should be taken at specific times during the day to maximize their effectiveness. Follow your doctor's instructions on when to take the medication.
4. Store the medication properly: Antibiotics should be stored in a cool, dry place away from direct sunlight. Keep them out of reach of children and pets.
5. Avoid cross-co

## B. FEW SHOT

In [16]:
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
  {
    "context": "People with neutropenia have an unusually low number of cells called neutrophils. Neutrophils are cells in your immune system that attack bacteria and other organisms when they invade your body. Neutrophils are a type of white blood cell. Your bone marrow creates these cells. They then travel in your bloodstream and move to areas of infection where they ingest and then neutralize the offending bacteria. Neutropenia itself often doesn't cause symptoms. In some cases, people only learn they have neutropenia when they have a blood test for an unrelated reason. It is most commonly seen--and even expected--as a result of chemotherapy used to treat cancer. But some people may have other symptoms from infection or the underlying problem causing the neutropenia. Infections can occur as a complication of neutropenia. They occur most often in the mucous membranes, such as the inside of the mouth and the skin. These infections can appear as: Ulcers Abscesses (collections of pus) Rashes Wounds that take a long time to heal Fever is also a common symptom of infection. In a neutropenic fever, it is common not to identify the exact cause, which is often normal gut bacteria that has made its way into the blood from weakened barriers. Neutropenic fevers are usually treated with antibiotics, even if an infectious source can't be identified. This is important because the weakened immune system means patients can get very sick very quickly. The risk for serious infection generally increases as: Neutrophil count goes down Duration of severe neutropenia gets longer The causes of neutropenia include: Problem in the production of neutrophils in the bone marrow Destruction of neutrophils outside the bone marrow Infection Nutritional deficiency Causes of decreased production of neutrophils include: Being born with a problem with bone marrow production (congenital) Leukemia and other conditions that affect the bone marrow or lead to bone marrow failure Radiation Chemotherapy Infections that can cause neutropenia include: Tuberculosis Dengue fever Viral infections such as Epstein- Barr virus, cytomegalovirus, HIV, viral hepatitis Increased destruction of neutrophils can be due to the body's immune system targeting neutrophils for destruction. This may be related to having an autoimmune condition, such as: Crohn's disease Rheumatoid arthritis Lupus In some people, neutropenia can be caused by certain medications, such as: Antibiotics Blood pressure drugs Psychiatric drugs Epilepsy drugs When deciding on treatment, health care providers consider the cause and severity of the neutropenia. Mild cases may not need any treatment. Approaches for treating neutropenia include: Antibiotics for fever. In neutropenic fever, the assumption is made that there is an infection causing the fever even when the source can't be found. A treatment called granulocyte colony-stimulating factor ( G- CSF). This stimulates the bone marrow to produce more white blood cells. It is used for several types of neutropenia, including low white cell count from chemotherpay. This treatment can be lifesaving in these cases. Changing medications, if possible, in cases of drug-induced neutropenia Granulocyte (white blood cell) transfusion (very uncommon) Stem cell transplants may be useful in treating some types of severe neutropenia, including those caused by bone marrow problems. People with neutropenia often need to take special steps to prevent infections. These neutropenia precautions include: Good hygiene, including frequent hand washing and good dental care, such as regular tooth brushing and flossing Avoiding contact with sick people Always wearing shoes Cleaning cuts and scrapes, then covering them with a bandage Using an electric shaver rather than a razor Avoiding animal waste and, when possible, not changing infants' diapers Avoiding unpasteurized dairy foods; undercooked meat; and raw fruits, vegetables, grains, nuts, and honey Staying out of hot tubs, ponds, and rivers"  ,
    "question": "What are some treatments for neutropenia?",
    "answer": "Neutrophils are cells in your immune system that attack bacteria and other organisms when they invade your body. This may be related to having an autoimmune condition, such as: Crohn's disease Rheumatoid arthritis Lupus In some people, neutropenia can be caused by certain medications, such as: Antibiotics Blood pressure drugs Psychiatric drugs Epilepsy drugs When deciding on treatment, health care providers consider the cause and severity of the neutropenia. Approaches for treating neutropenia include: Antibiotics for fever. A treatment called granulocyte colony-stimulating factor ( G- CSF). This stimulates the bone marrow to produce more white blood cells. Changing medications, if possible, in cases of drug-induced neutropenia Granulocyte (white blood cell) transfusion (very uncommon) Stem cell transplants may be useful in treating some types of severe neutropenia, including those caused by bone marrow problems."
  },
  {
    "context": "Migraine with brainstem aura or MBA (formerly known as basilar migraines) are headaches that start in the lower part of the brain, called the brainstem. They cause symptoms such as dizziness, double vision, and lack of coordination. These changes, called an aura, can happen about 10 minutes to 45 minutes before your head hurts. The headache pain of a basilar migraine often starts on one side of the head and then gradually spreads and gets stronger. This type of migraine can last anywhere from 4 to 72 hours. And it takes time to recover from one. You may feel drained for up to 24 hours after it's over. Migraines with brainstem aura are known by several different names: Basilar migraine Basilar artery migraine Basilar-type migraine Bickerstaff's syndrome Brainstem migraine Vertebrobasilar migraine MBA can affect people of all ages. Generally, though, they start in childhood or the teen years. Women are slightly more likely to have them than men. Triggers may include: Alcohol Stress Lack of sleep Some medications Hunger Female hormone changes Bright lights Caffeine Nitrites in some foods, like sandwich meat, bacon, and processed foods Overdoing physical activity Weather or altitude Symptoms differ for everyone, but some are typical: Nausea Vomiting Sensitivity to light and sound Cold hands or feet Dizziness Double vision or graying of vision Slurred speech or trouble speaking Temporary blindness Loss of balance Confusion Trouble hearing Body tingling Loss of consciousness Aura symptoms may last between 5 minutes and 1 hour. When the headache starts, you might feel an intense throbbing or pulsating pain on one or both sides of your head or sometimes at the back of your head. After you've had at least two attacks of at least two auras, your doctor often make the diagnosis of migraine with brainstem aura. The condition has many of the same symptoms as another type, called hemiplegic migraine. But the hemiplegic kind usually causes weakness of one side of the body or trouble speaking. Symptoms of MBA can also seem like the signs of other more serious conditions, like seizure disorders, stroke, meningitis, or brain tumors. To rule those out, you'll need to see a brain doctor, called a neurologist. He'll give you a thorough exam and ask you questions about your symptoms. He'll may also use tests like MRI, CT scans, and nerve tests to see what's causing your symptoms. Treatments for MBA generally aim to relieve symptoms of pain and nausea. You might take pain relievers such as acetaminophen, ibuprofen, and naproxen, as well as nausea medicines such as chlorpromazine, metoclopramide, and prochlorperazine. Your doctor may prescribe a medication that treats regular migraines, such as triptans. To keep from getting a MBA, it helps to avoid the things that usually cause one. Keep a journal of your attacks so you can figure out the things that trigger them. It also helps to live a healthy lifestyle. That means you need to: Get enough sleep. Limit your stress. Exercise daily. Diet can also affect migraines. Do these things: Eat a balanced diet. Avoid drugs and alcohol. Don't skip meals. Limit caffeine. Avoid any foods that have been triggers. Some common food triggers include: Dairy Wheat Chocolate Eggs Rye Tomatoes Oranges Along with these changes, If you don't respond to other treatments and you have 4 or more migraine days a month, your doctor may suggest preventive medicines. You can take these regularly to reduce the severity or frequency of the headaches. These include seizure medicines, blood pressure medicines (like beta blockers and calcium channel blockers), and some antidepressants. CGRP inhibitors are a new class of preventive medicine that your doctor may recommend if other medicines don't help.", 
    "question": "What are the symptoms of a migraine with brainstem aura?",
    "answer": "Triggers may include: Alcohol Stress Lack of sleep Some medications Hunger Female hormone changes Bright lights Caffeine Nitrites in some foods, like sandwich meat, bacon, and processed foods Overdoing physical activity Weather or altitude Symptoms differ for everyone, but some are typical: Nausea Vomiting Sensitivity to light and sound Cold hands or feet Dizziness Double vision or graying of vision Slurred speech or trouble speaking Temporary blindness Loss of balance Confusion Trouble hearing Body tingling Loss of consciousness Aura symptoms may last between 5 minutes and 1 hour."
  },
  {
    "context": "Migraine with brainstem aura or MBA (formerly known as basilar migraines) are headaches that start in the lower part of the brain, called the brainstem. They cause symptoms such as dizziness, double vision, and lack of coordination. These changes, called an aura, can happen about 10 minutes to 45 minutes before your head hurts. The headache pain of a basilar migraine often starts on one side of the head and then gradually spreads and gets stronger. This type of migraine can last anywhere from 4 to 72 hours. And it takes time to recover from one. You may feel drained for up to 24 hours after it's over. Migraines with brainstem aura are known by several different names: Basilar migraine Basilar artery migraine Basilar-type migraine Bickerstaff's syndrome Brainstem migraine Vertebrobasilar migraine MBA can affect people of all ages. Generally, though, they start in childhood or the teen years. Women are slightly more likely to have them than men. Triggers may include: Alcohol Stress Lack of sleep Some medications Hunger Female hormone changes Bright lights Caffeine Nitrites in some foods, like sandwich meat, bacon, and processed foods Overdoing physical activity Weather or altitude Symptoms differ for everyone, but some are typical: Nausea Vomiting Sensitivity to light and sound Cold hands or feet Dizziness Double vision or graying of vision Slurred speech or trouble speaking Temporary blindness Loss of balance Confusion Trouble hearing Body tingling Loss of consciousness Aura symptoms may last between 5 minutes and 1 hour. When the headache starts, you might feel an intense throbbing or pulsating pain on one or both sides of your head or sometimes at the back of your head. After you've had at least two attacks of at least two auras, your doctor often make the diagnosis of migraine with brainstem aura. The condition has many of the same symptoms as another type, called hemiplegic migraine. But the hemiplegic kind usually causes weakness of one side of the body or trouble speaking. Symptoms of MBA can also seem like the signs of other more serious conditions, like seizure disorders, stroke, meningitis, or brain tumors. To rule those out, you'll need to see a brain doctor, called a neurologist. He'll give you a thorough exam and ask you questions about your symptoms. He'll may also use tests like MRI, CT scans, and nerve tests to see what's causing your symptoms. Treatments for MBA generally aim to relieve symptoms of pain and nausea. You might take pain relievers such as acetaminophen, ibuprofen, and naproxen, as well as nausea medicines such as chlorpromazine, metoclopramide, and prochlorperazine. Your doctor may prescribe a medication that treats regular migraines, such as triptans. To keep from getting a MBA, it helps to avoid the things that usually cause one. Keep a journal of your attacks so you can figure out the things that trigger them. It also helps to live a healthy lifestyle. That means you need to: Get enough sleep. Limit your stress. Exercise daily. Diet can also affect migraines. Do these things: Eat a balanced diet. Avoid drugs and alcohol. Don't skip meals. Limit caffeine. Avoid any foods that have been triggers. Some common food triggers include: Dairy Wheat Chocolate Eggs Rye Tomatoes Oranges Along with these changes, If you don't respond to other treatments and you have 4 or more migraine days a month, your doctor may suggest preventive medicines. You can take these regularly to reduce the severity or frequency of the headaches. These include seizure medicines, blood pressure medicines (like beta blockers and calcium channel blockers), and some antidepressants. CGRP inhibitors are a new class of preventive medicine that your doctor may recommend if other medicines don't help.",
    "question": "Can migraine with brainstem aura be prevented?",
    "answer": "To keep from getting a MBA, it helps to avoid the things that usually cause one. Keep a journal of your attacks so you can figure out the things that trigger them. It also helps to live a healthy lifestyle. That means you need to: Get enough sleep."
  }
]

In [17]:

example_prompt = PromptTemplate(
    input_variables=["context", "question", "answer"],
    template="Context: {context}\nQuestion: {question}\nAnswer: {answer}"
)


In [18]:

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)


In [19]:
%%time
result = llm(prompt.format(input="What types of exercise are best for people with asthma?"))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: Asthma is a chronic lung condition that makes it difficult to breathe. Regular exercise can improve lung function and overall fitness, but it's important to choose activities that won't worsen asthma symptoms. Low-impact exercises, such as swimming, cycling, and walking, are generally recommended for people with asthma. Swimming is a great option because it's a low-impact aerobic exercise that can help improve cardiovascular fitness and lung function. Cycling is another low-impact exercise that can be done indoors or outdoors. Walking is also a low-impact exercise that can be done indoors or outdoors. It's important to talk to your doctor or an asthma specialist before starting any new exercise routine. They can help you determine the best type of exercise for your individual needs and provide guidance on how to manage asthma symptoms while exercising.
CPU times: user 18.4 s, sys: 228 ms, total: 18.6 s
Wall time: 18.6 s


In [20]:
%%time
result = llm(prompt.format(input="How is obsessive-compulsive disorder diagnosed?"))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: Obsessive-compulsive disorder (OCD) is a mental illness characterized by intrusive thoughts (obsessions) and repetitive behaviors (compulsions) that a person feels driven to perform. Diagnosing OCD typically involves a combination of self-report questionnaires, clinical interviews, and medical history.

Self-report questionnaires are standardized forms that assess various aspects of psychological functioning. For example, the Yale-Brown Obsessive Compulsive Scale (YBOCS) is a widely used questionnaire designed specifically to measure the severity of OCD symptoms.

Clinical interviews involve a face-to-face conversation with a mental health professional who asks specific questions about the individual's thoughts, feelings, and behaviors. During this interview, the clinician will try to understand the nature and extent of the individual's obsessions and compulsions, as well as how they impact their daily life.

Medical history is also important in diagnosing OCD. The clinician w

In [21]:
%%time
result = llm(prompt.format(input="When are you more likely to get a blood clot?"))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: Blood clots are more likely to form when blood flow is slowed or blocked. This can happen in several ways:

* During pregnancy, especially in the third trimester, when blood vessels stretch and become less elastic.
* After surgery, especially orthopedic surgery, when immobility can slow blood flow.
* When taking certain medications, such as birth control pills, antiplatelet drugs, and corticosteroids.
* When smoking, since nicotine constricts blood vessels and reduces blood flow.
* When obese, since excess weight can put additional strain on blood vessels and increase the risk of clots.
* When sedentary, since lack of exercise can slow blood flow and increase the risk of clots.
* When experiencing trauma, such as a car accident or sports injury, which can damage blood vessels and increase the risk of clots.
* When traveling long distances, especially by air, where the reduced oxygen levels and dehydration can increase the risk of clots.
* When consuming alcohol, since it can t

In [22]:
%%time
result = llm(prompt.format(input="How should you lift objects to prevent back pain?"))
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: Here are some tips to help you lift objects safely and prevent back pain:

1. Plan ahead: Before lifting, assess the weight and size of the object, and determine how you will lift it.
2. Use proper lifting techniques: Bend your knees and hips, keep your back straight, and lift with your legs, not your back.
3. Hold the object close to your body: Keep the object as close to your body as possible to reduce strain on your back muscles.
4. Take breaks: If you need to lift a heavy object, take breaks frequently to rest and stretch your back.
5. Use assistive devices: Consider using tools such as dollies, carts, or straps to help lift heavy objects.
6. Practice good posture: Stand tall with your shoulders back and your chin up to maintain good posture while lifting.
7. Maintain a healthy weight: Carrying excess weight can put additional strain on your back muscles, leading to pain and discomfort.
8. Exercise regularly: Regular exercise can strengthen your back muscles and improve fl

In [23]:
%%time
result = llm(prompt.format(input="How can you be smart with antibiotics?"))
print(result)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: Antibiotics are powerful tools that can save lives. However, they should be used wisely to ensure their effectiveness and minimize potential harm. Here are some ways to be smart with antibiotics:

1. Take antibiotics exactly as directed by your healthcare provider. Follow the dosage instructions carefully and complete the full course of treatment.
2. Use antibiotics only when necessary. Antibiotics should be reserved for situations where other treatments are not effective or safe.
3. Preserve the effectiveness of antibiotics by completing the full course of treatment. Taking only a partial course of antibiotics can contribute to the development of antibiotic-resistant bacteria.
4. Practice good hygiene to prevent the spread of germs. Wash your hands frequently, cover your mouth and nose when coughing or sneezing, and clean surfaces regularly.
5. Be cautious when using antibiotics to treat minor illnesses. Antibiotics should only be used to treat bacterial infections, not viral

## C. CHAIN OF THOUGHTS

In [24]:
from langchain.chains import LLMChain

template = """
[INST] <>
Act as a Multiple Answer Spans Healthcare Question Answering helpful assistant and answer the user's questions in details with reasoning. Do not give any false information. In case you don't have answer, specify why the question can't be answered.
<>

{question} [/INST]
"""

examples_prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)
# Initialize LLMChain with the model and tokenizer
llm_chain = LLMChain(llm=llm, prompt=examples_prompt)

# Generate chain of thoughts using the provided prompt
input_text = "What types of exercise are best for people with asthma?"
chain_of_thoughts = llm_chain.run(input_text)

print(chain_of_thoughts)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


There are several types of exercises that can be beneficial for people with asthma. However, it is important to note that everyone's body is different, so it is always best to consult with a healthcare professional before starting any new exercise routine. Here are some types of exercises that may be recommended for people with asthma:

1. Low-impact aerobic exercises: These include activities such as swimming, cycling, or walking on flat surfaces. These exercises can help improve cardiovascular fitness without putting too much stress on the lungs.
2. Resistance training: Strength training exercises can help build muscle mass and improve overall physical function. This type of exercise can also help reduce symptoms of asthma by improving lung capacity and reducing inflammation.
3. Yoga and tai chi: These mind-body practices can help improve flexibility, balance, and strength while also promoting relaxation and reducing stress levels. They can also help improve breathing techniques, whi

In [25]:
# Generate chain of thoughts using the provided prompt
input_text = "How is obsessive-compulsive disorder diagnosed?"
chain_of_thoughts = llm_chain.run(input_text)

print(chain_of_thoughts)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Obsessive-compulsive disorder (OCD) is typically diagnosed through a combination of self-reporting symptoms, clinical interview, and sometimes psychological testing. During the diagnostic process, healthcare providers will assess the severity and duration of the individual's symptoms, as well as their impact on daily life. They may also look for evidence of other mental health conditions that could be co-occurring with OCD.

Self-reporting symptoms: The first step in diagnosing OCD is usually to ask the person about their symptoms. This involves asking them to describe their thoughts, feelings, and behaviors related to their obsessions and compulsions.

Clinical interview: A clinical interview is a structured conversation between the patient and the healthcare provider during which they discuss the patient's symptoms in more detail. The interview helps the healthcare provider to identify patterns and triggers associated with the patient's obsessions and compulsions.

Psychological test

In [26]:

# Generate chain of thoughts using the provided prompt
input_text = "When are you more likely to get a blood clot?"
chain_of_thoughts = llm_chain.run(input_text)

print(chain_of_thoughts)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


A blood clot is a blockage of blood flow that can occur in various parts of the body. There are several factors that can increase your risk of getting a blood clot:

1. Age: The older you are, the higher your risk of developing a blood clot. This is because as we age, our blood vessels become stiffer and less flexible, making it harder for blood to flow smoothly.
2. Family history: If someone in your family has had a blood clot, you may be at increased risk of developing one yourself.
3. Medical conditions: Certain medical conditions, such as heart disease, stroke, diabetes, high blood pressure, and cancer, can increase your risk of getting a blood clot.
4. Smoking: Smoking damages the lining of your arteries, which can make them more prone to forming blood clots.
5. Obesity: Being overweight or obese can also increase your risk of developing a blood clot.
6. Sedentary lifestyle: Leading a sedentary lifestyle, without regular physical activity, can also increase your risk of getting a 

In [27]:

# Generate chain of thoughts using the provided prompt
input_text = "How should you lift objects to prevent back pain?"
chain_of_thoughts = llm_chain.run(input_text)

print(chain_of_thoughts)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



There are several ways to lift objects safely to prevent back pain:

1. Bend your knees and hips instead of bending at the waist or back. This will help keep your spine straight and reduce strain on your lower back muscles.
2. Keep your weight evenly distributed between both legs. Avoid lifting with one leg, as this puts extra stress on your back.
3. Hold the object close to your body, as far away from your arms as possible. This reduces the amount of force needed to lift the object and helps keep your back straight.
4. Use proper lifting techniques, such as squatting down and pushing up with your legs rather than straining your back.
5. Take breaks if necessary, especially if you are lifting heavy objects. Resting for short periods of time can help prevent muscle fatigue and reduce the risk of injury.
6. If you experience pain while lifting, seek medical attention immediately. It may be necessary to modify your lifting technique or seek physical therapy to address any underlying issu

In [28]:
# Generate chain of thoughts using the provided prompt
input_text = "How can you be smart with antibiotics?"
chain_of_thoughts = llm_chain.run(input_text)

print(chain_of_thoughts)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Antibiotics are powerful medications that can help fight bacterial infections. However, it is important to use them correctly to avoid resistance and other potential complications. Here are some ways to be smart with antibiotics:

1. Follow your doctor's instructions carefully: Your doctor will prescribe the right type of antibiotic for your infection and tell you how long to take it. Make sure to follow their instructions exactly as they give them.
2. Take all of the medication: It is important to complete the full course of antibiotics even if you start to feel better before finishing. This helps prevent bacteria from developing resistance to the antibiotic.
3. Don't share your medication: Antibiotics should only be taken by the person who has been prescribed them. Sharing medication can lead to resistance and other complications.
4. Avoid taking antibiotics unnecessarily: Antibiotics should only be used when necessary to treat an infection. Taking them unnecessarily can contribute 